In [21]:
import torch
import numpy as np

In [42]:
raw_preds = torch.rand(2, 3, 25, 25)
y_true = torch.argmax(torch.rand(2, 3, 25, 25))
num_ranges = 15

In [28]:
probs = torch.softmax(raw_preds, dim=1)

# predictions are argmax predictions
# confidences are the probabilities of the respective predictions
confidences, y_hat = torch.max(probs, dim=1)

classes = raw_preds.shape[1]


In [36]:
def get_batch(batch_idx, y_true, y_hat, confidences):
    y = y_true[batch_idx, :, :, :]
    y_hat = y_hat[batch_idx, :, :, :] 
    confidence = confidences[batch_idx, :, :, :]

    return y, y_hat, confidence

1875

In [44]:
def sort_probs(confidence):
    flat = torch.flatten(confidence)
    sorted_one_hot, indices = torch.sort(flat)

    return sorted_one_hot

In [45]:
def get_accuracy(class_idx, upper, lower, y, confidence_values, pred_values):
    
    # FIND PREDICTED VALUES IN THE RANGE
    # filter confidence and argmaxed predictions for values in the range of the bin
    # also, return the indices where these filtered values are in the matrix
    fil_confidence = torch.where(y == class_idx, confidence_values, 0)
    fil_pred_values = torch.where((fil_confidence >= lower) & (fil_confidence <= upper), pred_values, torch.tensor([]))
    fil_idxs = fil_confidence.where((fil_confidence >= lower) & (fil_confidence <= upper))

    # FIND GROUND TRUTH VALUES IN THE RANGE
    fil_truth = y[fil_idxs]

    # CALCULATE THE ACCURACY FOR THIS RANGE FOR THE GIVEN CLASS
    diff = fil_pred_values - fil_truth
    num_correct = diff.size() - torch.count_nonzero(diff)
    acc = torch.abs(num_correct / fil_truth.size())


    return acc

In [41]:
batch_size = raw_preds.shape[0]
# make handling batches happen in a function
for batch_idx in range(batch_size):
    y, y_hat, confidence = get_batch(batch_idx, y_true, y_hat, confidences)

    # number of values for each class
    N = y.size[-2] * y.size[-1]

    # sum of calibration errors for every range for every class
    sum = 0

    for class_idx in range(classes):
        # find total range of probabilities of class in image and sort
        sort = sort_probs(confidence)

        # split into ranges
        ranges = torch.linspace(0, N, (N/num_ranges))

        prev_r = 0
        for r in ranges:
            upper = r
            lower = prev_r

            # calculate confidence and accuracy
            acc = get_accuracy(class_idx, upper, lower, y, confidence, y_hat)
            conf = torch.mean(sort)

            # difference for this range
            cal_error = torch.abs(acc - conf)
            sum+=cal_error
            
            prev_r = r






sorted_one_hot.shape

torch.Size([1250])